# `pandas` part 1

This week we learn about the following five areas of pandas:

- Data I/O
- Understanding the `DataFrame` and `Series`
- Indexing and Slicing
- First look at the data
- Summary functions

## Importing Libraries

In python, we use the `import` statement to "load" code from other libraries. This is similar to the `library()` function in `R`.

Note we can also use `import library as lib` to rename the imported library. Although it makes for less typing, I recommend sticking to widely accepted abbreviations.

In [1]:
import pandas as pd

pd.__version__ # you can check the version of any library with this command
               # if you get anything older than 0.25.0, try updating pandas with: conda update pandas

'0.25.3'

# Understanding the `DataFrame` and `Series`

## `pd.Series`

To pull up the documentation on a class, method or module, use the `?` magic.

In [50]:
pd?

In [51]:
pd.Series?

# Reading in Data

For this lecture we use the BES data. Relevant data files can be downloaded here: **insert link**

The original file was in `dta` format, but I've saved it in a number of formats.
Here are the file names with their associated sizes in kilobytes.

```
360K    bes_data_subset_week2.csv
296K    bes_data_subset_week2.feather
636K    bes_data_subset_week2.json
```

Let's just use the `feather` format for now, as this is the easiest to work with.

In [33]:
df = pd.read_feather("bes_data_subset_week2.feather")

We can view the first/last 30 rows (and 20 columns) by just writing the name of the dataframe.

In [36]:
df

,finalserialno,a01,a02,a03,region,recontact,agency,Constit_Code,Constit_Name,Interview_Date,total_num_dwel,total_num_hous,num_elig_people,edlevel,turnoutValidationReg
0,10115,nhs,Labour,Fairly interested,East Midlands,No,GfK,Ashfield,E14000535,06/09/2017,1,1,2,No qualifications,Voted
1,10119,brexit,None/No party,Fairly interested,East Midlands,No,GfK,Ashfield,E14000535,06/09/2017,1,1,1,Below GCSE,Voted
2,10125,society,Don`t know,Fairly interested,East Midlands,No,GfK,Ashfield,E14000535,06/09/2017,1,1,1,No qualifications,NaN
3,10215,immigration,Don`t know,Fairly interested,East Midlands,No,GfK,Ashfield,E14000535,24/08/2017,1,1,2,No qualifications,Voted
4,10216,brexit,Labour,Fairly interested,East Midlands,No,GfK,Ashfield,E14000535,24/08/2017,1,1,2,Undergraduate,Voted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2189,69923,TERRORISM - YOU SEE THE NEWS AND IT'S FRIGHTEN...,Don`t know,Fairly interested,Yorkshire & Humber,No,Kantar,York Outer,E14001062,20/09/2017,1,1,2,GCSE,NaN
2190,69925,brexit,None/No party,Not very interested,Yorkshire & Humber,No,Kantar,York Outer,E14001062,31/07/2017,1,1,1,Undergraduate,NaN
2191,70016,BREXIT - HOW IT WILL REDUCE THE NUMBERS OF PEO...,Don`t know,Not very interested,Yorkshire & Humber,No,Kantar,York Outer,E14001062,19/09/2017,1,1,1,Undergraduate,Voted
2192,70022,brexit,None/No party,Not very interested,Yorkshire & Humber,No,Kantar,York Outer,E14001062,02/08/2017,1,1,1,No qualifications,NaN


# 